In [94]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [95]:
train_data = pd.read_csv('C:\\Users\\SIDDU\\OneDrive\\Desktop\\work\\selflearnml\\titanic\\train.csv')
test_data = pd.read_csv('C:\\Users\\SIDDU\\OneDrive\\Desktop\\work\\selflearnml\\titanic\\test.csv')
valid_data = pd.read_csv('C:\\Users\\SIDDU\\OneDrive\\Desktop\\work\\selflearnml\\titanic\\gender_submission.csv')

In [96]:
# normal method
X_train = train_data.drop(["Survived","Name","Cabin","Ticket"],axis=1)
y_train = train_data.Survived
X_test = test_data.drop(["Name","Cabin","Ticket"],axis=1)
y_test = valid_data.drop(["PassengerId"],axis=1)
cat_cols = [col for col in X_train.columns if X_train[col].dtypes == 'O']
num_cols = [col for col in X_train.columns if X_train[col].dtypes == 'int64' or X_train[col].dtypes == 'float64']
imp = KNNImputer()
X_train_imputed = imp.fit_transform(X_train[num_cols])
X_test_imputed = imp.transform(X_test[num_cols])
ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_train_encoded = ohe.fit_transform(X_train[cat_cols])
X_test_encoded = ohe.transform(X_test[cat_cols])
X_train_final = np.append(X_train_imputed,X_train_encoded,axis=1)
X_test_final = np.append(X_test_imputed,X_test_encoded,axis=1)
model = RandomForestClassifier(n_estimators=100,random_state=0)
model.fit(X_train_final,y_train)
predictions = model.predict(X_test_final)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

0.8133971291866029


In [97]:
# Using pipelines and column transformers
X_train = train_data.drop(["Survived","Name","Cabin","Ticket"],axis=1)
y_train = train_data.Survived
X_test = test_data.drop(["Name","Cabin","Ticket"],axis=1)
y_test = valid_data.drop(["PassengerId"],axis=1)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
pl1 = Pipeline([
    ("step1", KNNImputer()),
    ("step2", StandardScaler()),
])
pl2 = Pipeline([
    ("step1", SimpleImputer(strategy='constant')),
    ("step2", OneHotEncoder(sparse=False, handle_unknown='ignore')),
])
cat_cols = [col for col in X_train.columns if X_train[col].dtypes == 'O']
num_cols = [col for col in X_train.columns if X_train[col].dtypes == 'int64' or X_train[col].dtypes == 'float64']
ct = ColumnTransformer([
    ("step1",pl1,num_cols),
    ("step2",pl2,cat_cols),
])
final_pl = Pipeline([
    ("step1",ct),
    ("step2",RandomForestClassifier(n_estimators=100,random_state=0)),
])
final_pl.fit(X_train,y_train)
predictions2 = final_pl.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions2))

0.8133971291866029


In [98]:
# Using cross validation 
X_train = train_data.drop(["Survived","Name","Cabin","Ticket"],axis=1)
y_train = train_data.Survived
X_test = test_data.drop(["Name","Cabin","Ticket"],axis=1)
y_test = valid_data.drop(["PassengerId"],axis=1)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
pl1 = Pipeline([
    ("step1", KNNImputer()),
    ("step2", StandardScaler()),
])
pl2 = Pipeline([
    ("step1", SimpleImputer(strategy='constant')),
    ("step2", OneHotEncoder(sparse=False, handle_unknown='ignore')),
])
cat_cols = [col for col in X_train.columns if X_train[col].dtypes == 'O']
num_cols = [col for col in X_train.columns if X_train[col].dtypes == 'int64' or X_train[col].dtypes == 'float64']
ct = ColumnTransformer([
    ("step1",pl1,num_cols),
    ("step2",pl2,cat_cols),
])
final_pl = Pipeline([
    ("step1",ct),
    ("step2",RandomForestClassifier(n_estimators=100,random_state=0)),
])
from sklearn.model_selection import cross_val_score
cvs = cross_val_score(final_pl,X_train,y_train,cv=5,scoring='accuracy')
print(cvs.mean())
final_pl.fit(X_train,y_train)
predictions2 = final_pl.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions2))

0.8048145125855252
0.8133971291866029
